In [ ]:
#trading on alpaca
import alpaca_trade_api as tradeapi
import time
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# API key and secret
API_KEY = ''
API_SECRET = ''
BASE_URL = 'https://paper-api.alpaca.markets'

# API connection
api = tradeapi.REST(API_KEY, API_SECRET, base_url = BASE_URL, api_version = 'v2')

# storage path
path = "/Users/kaiwentay/Documents/GitHub/Pair_Trading_Fun/Data"

# Specify the asset symbol and trading interval
symbol1 = 'SOXL'
symbol2 = 'SOXX'
interval = 5  # Time interval in seconds
wealth = 100000
beta = 7
multiplier = 0.5
mean_spread = 341.38278004873126
std_spread = 2.4724391810179016

# Main trading loop
while True:
    try:
        # Get historical market data for the specified symbol and interval
        historical_data1 = api.get_bars(symbol1, '1Min', limit=1).df[symbol1]
        historical_data2 = api.get_bars(symbol2, '1Min', limit=1).df[symbol2]
        
        # Get the last close price
        stock_X_price = historical_data1['close'].iloc[-1]
        stock_Y_price = historical_data2['close'].iloc[-1]
        spread = stock_Y_price['close'] - beta * stock_X_price['close']

        position = None  # 'long', 'short', or None
        entry_price = None  # Entry price
        max_portfolio_value = wealth * 0.5  # Maximum amount to be used for trading
        orderbook = pd.DataFrame(columns=['Date', 'Order', 'Price', 'units of X', 'units of Y', 'price of X', 'Price of Y'])

        # Determine whether to place a Buy or Sell order based on alternating intervals
        if position is None:
            units = int(max_portfolio_value / abs(stock_Y_price - beta * stock_X_price))
            if spread > mean_spread + multiplier * std_spread:
                position = 'short'
                entry_price = spread
                orderbook = orderbook.append({'Date': datetime.datetime.now(), 'Order': 'short', 'Price of X': stock_X_price, 'Price of Y': stock_Y_price, 'units of X': beta * units, 'units of Y': -units}, ignore_index=True)
                api.submit_order(
                    symbol = symbol2,
                    qty = units,  # Adjust quantity as needed
                    side = 'sell',
                    type = 'limit',
                    time_in_force = 'gtc',
                    limit_price = stock_Y_price * 0.99,  # Place a limit order slightly below the current price
                )
                print("Sell order placed at {}".format(stock_Y_price))
                api.submit_order(
                    symbol = symbol1,
                    qty = beta * units,  # Adjust quantity as needed
                    side = 'buy',
                    type = 'limit',
                    time_in_force = 'gtc',
                    limit_price = stock_X_price * 1.01,  # Place a limit order slightly above the current price
                )
                print("Buy order placed at {}".format(stock_X_price))
            elif spread < mean_spread - multiplier * std_spread:
                position = 'long'
                entry_price = spread
                orderbook = orderbook.append({'Date': datetime.datetime.now(), 'Order': 'long', 'Price of X': stock_X_price, 'Price of Y': stock_Y_price, 'units of X': -beta * units, 'units of Y': units}, ignore_index=True)
                api.submit_order(
                    symbol = symbol2,
                    qty = units,
                    side = 'buy',
                    type = 'limit',
                    time_in_force = 'gtc',
                    limit_price = stock_Y_price * 1.01,  # Place a limit order slightly above the current price
                )
                print("Buy order placed at {}".format(stock_Y_price))
                api.submit_order(
                    symbol = symbol1,
                    qty = beta * units,
                    side = 'sell',
                    type = 'limit',
                    time_in_force = 'gtc',
                    limit_price = stock_X_price * 0.99,  # Place a limit order slightly below the current price
                )
                print("Sell order placed at {}".format(stock_X_price))
        else:
            # Check for exit signal
            units = abs(orderbook['units of Y'].iloc[-1])
            if (position == 'short' and current_spread <= mean_spread):
                # Close position
                exit_price = spread
                pnl = (entry_price - exit_price) * units
                wealth += pnl
                orderbook = orderbook.append({'Date': datetime.datetime.now(), 'Order': 'close', 'Price of X': stock_X_price, 'Price of Y': stock_Y_price, 'units of X': -beta * units, 'units of Y': units}, ignore_index=True)
                api.submit_order(
                    symbol = symbol2,
                    qty = units,
                    side = 'buy',
                    type = 'limit',
                    time_in_force = 'gtc',
                    limit_price = stock_Y_price * 1.01,  # Place a limit order slightly above the current price
                )
                print("Buy order placed at {}".format(stock_Y_price))
                api.submit_order(
                    symbol = symbol1,
                    qty = beta * units,
                    side = 'sell',
                    type = 'limit',
                    time_in_force = 'gtc',
                    limit_price = stock_X_price * 0.99,  # Place a limit order slightly below the current price
                )
                print("Sell order placed at {}".format(stock_X_price))
                position = None
                entry_price = None
            elif (position == 'long' and current_spread >= mean_spread):
                # Close position
                exit_price = spread
                pnl = (exit_price - entry_price) * units
                wealth += pnl
                orderbook = orderbook.append({'Date': datetime.datetime.now(), 'Order': 'close', 'Price of X': stock_X_price, 'Price of Y': stock_Y_price, 'units of X': beta * units, 'units of Y': -units}, ignore_index=True)
                api.submit_order(
                    symbol = symbol2,
                    qty = units,
                    side = 'sell',
                    type = 'limit',
                    time_in_force = 'gtc',
                    limit_price = stock_Y_price * 0.99,  # Place a limit order slightly below the current price
                )
                print("Sell order placed at {}".format(stock_Y_price))
                api.submit_order(
                    symbol = symbol1,
                    qty = beta * units,
                    side = 'buy',
                    type = 'limit',
                    time_in_force = 'gtc',
                    limit_price = stock_X_price * 1.01,  # Place a limit order slightly above the current price
                )
                print("Buy order placed at {}".format(stock_X_price))
                position = None
                entry_price = None

        # Wait for the next interval
        time.sleep(interval)

    except Exception as e:
        print(f"An error occurred: {e}")